In [10]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
from scipy import integrate
from mpl_toolkits.mplot3d import Axes3D

from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPool2D, LSTM
from keras import optimizers
from keras.layers import Activation
from keras.utils.generic_utils import get_custom_objects
from keras import backend as K

In [2]:
import os
import scipy.io
import scipy.misc
import skimage
from skimage.transform import resize
plt.rcParams['figure.figsize'] = [10, 10]
plt.rcParams.update({'font.size': 18})

mat_contents = scipy.io.loadmat(os.path.join('/Users/zhhh/databook_python/DATA/VORTALL.mat'))
X=mat_contents['VORTALL'].T

In [3]:
nn_input = X[:100,:]
nn_output = X[1:101,:]

## feedforward NN

In [5]:
model = Sequential()

#add model layers
model.add(Dense(1024, activation='sigmoid'))
model.add(Dense(512, activation='relu'))
model.add(Dense(64, activation='linear'))
model.add(Dense(512, activation='linear'))
model.add(Dense(1024, activation='linear'))
model.add(Dense(89351, activation='linear'))

sgd_optimizer = optimizers.SGD(momentum=0.9)
model.compile(optimizer='adam', loss='mse')
History = model.fit(nn_input, nn_output, epochs=100)

Epoch 1/100
4/4 [==============================] - 10s 1s/step - loss: 1.2370
Epoch 2/100
4/4 [==============================] - 4s 854ms/step - loss: 0.6617
Epoch 3/100
4/4 [==============================] - 4s 937ms/step - loss: 0.4616
Epoch 4/100
4/4 [==============================] - 4s 975ms/step - loss: 0.3893
Epoch 5/100
4/4 [==============================] - 4s 906ms/step - loss: 0.3379
Epoch 6/100
4/4 [==============================] - 4s 917ms/step - loss: 0.3037
Epoch 7/100
4/4 [==============================] - 4s 1s/step - loss: 0.2941
Epoch 8/100
4/4 [==============================] - 3s 846ms/step - loss: 0.2902
Epoch 9/100
4/4 [==============================] - 4s 876ms/step - loss: 0.2388
Epoch 10/100
4/4 [==============================] - 3s 846ms/step - loss: 0.1816
Epoch 11/100
4/4 [==============================] - 3s 865ms/step - loss: 0.1561
Epoch 12/100
4/4 [==============================] - 4s 861ms/step - loss: 0.1373
Epoch 13/100
4/4 [========================

In [ ]:
y_predict = np.zeros((51, 89351))
y_predict[0,:] = X[102,:]
for j in range(50):
    y_predict[j+1,:] = model.predict(np.reshape(y_predict[j,:],(1,89351)))
y_predict = y_predict.T

In [ ]:
n_vortall = np.shape(y_predict)[1]

pred = np.reshape(y_predict, (449, 199, n_vortall))

test_path = '/Users/zhhh/databook_python/CH06/fig_test/'
for i in range(np.shape(pred)[2]):
    skimage.io.imsave(test_path + str(i) + '.png', pred[:,:,i])


In [8]:
import imageio
#plot flow
def plotFlow(X):    
    ## Plot Mode 2
    vortmin = -5
    vortmax = 5
    V2 = np.copy(np.real(np.reshape(X,(449,199))))
    V2 = V2.T

    # normalize values... not symmetric
    minval = np.min(V2)
    maxval = np.max(V2)

    if np.abs(minval) < 5 and np.abs(maxval) < 5:
        if np.abs(minval) > np.abs(maxval):
            vortmax = maxval
            vortmin = -maxval
        else:
            vortmin = minval
            vortmax = -minval

    V2[V2 > vortmax] = vortmax
    V2[V2 < vortmin] = vortmin

    plt.imshow(V2,cmap='jet',vmin=vortmin,vmax=vortmax)

    cvals = np.array([-4,-2,-1,-0.5,-0.25,-0.155])
    plt.contour(V2,cvals*vortmax/5,colors='k',linestyles='dashed',linewidths=1)
    plt.contour(V2,np.flip(-cvals)*vortmax/5,colors='k',linestyles='solid',linewidths=0.4)

    plt.scatter(49,99,5000,color='k') # draw cylinder

    #plt.show()
#create movie
def createMovie(X):
    plt.rcParams['figure.figsize'] = [8, 8]
    filenames = []
    for i in range(50):
        plotFlow(X[:,i])
        filename = './fig_test/'+str(i)+'.png'
        filenames.append(filename)
        plt.savefig(filename)
        plt.close()
    with imageio.get_writer('vort_pred.gif', mode='I') as writer:
        for filename in filenames:
            image = imageio.v2.imread(filename)
            writer.append_data(image)
    for filename in set(filenames):
        os.remove(filename)

In [9]:
from IPython.display import Image
createMovie(y_predict)
Image(url='vort_pred.gif')  


## LSTM NN

In [ ]:
sequence_size = 10
train_size = 100
test_size = 50
rnn_input = np.zeros((train_size*151-sequence_size-1),sequence_size, 89351)
rnn_output = np.zeros((train_size*151-sequence_size-1),sequence_size, 89351)
for k in range(151-sequence_size-1):
    rnn_input[(151-sequence_size-1) + k, :] = 

In [ ]:
model_LSTM = Sequential()
model.add(LSTM(16,input_shape = (None, 89351)))
model.add(Dense(89351))
sgd = optimizers.SGD(0.01)
model.compile(optimizer = sgd, loss = 'mean_square_error')
model.fit(rnn_input, rnn_output, epochs = 20)